In [1]:
from matplotlib import pyplot
from matplotlib.image import imread
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import load 
import numpy as np
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

Using TensorFlow backend.
C:\Users\nidhimh\Anaconda3\envs\Tensor150\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nidhimh\Anaconda3\envs\Tensor150\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nidhimh\Anaconda3\envs\Tensor150\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nidhimh\Anaconda3\en

In [39]:
# baseline model for the dogs vs cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['acc'], color='blue', label='train')
    pyplot.plot(history.history['val_acc'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('Train_PreProcessed/train/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory('Train_PreProcessed/test/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=20)
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it))
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Found 1815 images belonging to 2 classes.
Found 581 images belonging to 2 classes.
Epoch 1/20
29/29 [==============================] - ETA: 13s - loss: 1.1106 - acc: 0.52 - 728s 25s/step - loss: 1.0965 - acc: 0.5209 - val_loss: 0.6789 - val_acc: 0.5232
Epoch 2/20
29/29 [==============================] - 347s 12s/step - loss: 0.6579 - acc: 0.6013 - val_loss: 0.6063 - val_acc: 0.6403
Epoch 3/20
29/29 [==============================] - 218s 8s/step - loss: 0.5601 - acc: 0.7295 - val_loss: 0.4852 - val_acc: 0.8107
Epoch 4/20
29/29 [==============================] - 198s 7s/step - loss: 0.5007 - acc: 0.7577 - val_loss: 0.5423 - val_acc: 0.6695
Epoch 5/20
29/29 [==============================] - 172s 6s/step - loss: 0.4623 - acc: 0.7758 - val_loss: 0.3689 - val_acc: 0.8348
Epoch 6/20
29/29 [==============================] - 192s 7s/step - loss: 0.3571 - acc: 0.8631 - val_loss: 0.2782 - val_acc: 0.9243
Epoch 7/20
29/29 [==============================] - 209s 7s/step - loss: 0.2833 - acc: 0.90

In [14]:
# baseline model for the dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['acc'], color='blue', label='train')
    pyplot.plot(history.history['val_acc'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('Train_PreProcessed/train/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory('Train_PreProcessed/test/',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=25)
    # save model 
    model.save('glassesOrNo.h5')
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it))
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Found 1815 images belonging to 2 classes.
Found 581 images belonging to 2 classes.
Epoch 1/20
29/29 [==============================] - 164s 6s/step - loss: 1.0184 - acc: 0.5231 - val_loss: 0.6941 - val_acc: 0.4802
Epoch 2/20
29/29 [==============================] - 135s 5s/step - loss: 0.6819 - acc: 0.5567 - val_loss: 0.6883 - val_acc: 0.5886
Epoch 3/20
29/29 [==============================] - 137s 5s/step - loss: 0.6777 - acc: 0.5746 - val_loss: 0.6855 - val_acc: 0.5904
Epoch 4/20
29/29 [==============================] - 156s 5s/step - loss: 0.6690 - acc: 0.5869 - val_loss: 0.6763 - val_acc: 0.6713
Epoch 5/20
29/29 [==============================] - 153s 5s/step - loss: 0.6464 - acc: 0.6333 - val_loss: 0.6652 - val_acc: 0.6540
Epoch 6/20
29/29 [==============================] - 142s 5s/step - loss: 0.6334 - acc: 0.6311 - val_loss: 0.6591 - val_acc: 0.6558
Epoch 7/20
29/29 [==============================] - 132s 5s/step - loss: 0.6302 - acc: 0.6518 - val_loss: 0.6589 - val_acc: 0.6437


In [27]:
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

In [20]:
from keras.models import load_model

In [21]:
model = load_model('glassesOrNo.h5')

In [36]:
img = load_image("C:\\Users\\nidhimh\\Documents\\Glasses\\Test\\face-4932.png")

In [37]:
# predict the class
result = model.predict(img)
print(result[0])

[1.]


In [38]:
# save model weights
model.save_weights('glassorno_model_weights.h5')

In [ ]:
# glasses -> 1
# no glasses -> 0